In [8]:
pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.7 MB/s eta 0:00:00


In [9]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.oxml import OxmlElement
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

# Load the Excel file into a DataFrame
df = pd.read_excel('/content/Classes.xlsx')

# Create a new Word document
doc = Document()

# Add a title to the document
doc.add_heading('Classes Overview', 0)

# Add a table to the document. The number of rows is one more than the DataFrame (for headers),
# and the number of columns is the same as in the DataFrame.
table = doc.add_table(rows=df.shape[0]+1, cols=df.shape[1])
table.style = 'Table Grid'  # Apply a basic grid style to the table

# Define the function to set cell background color
def set_cell_background(cell, fill):
    shading_elm = parse_xml(r'<w:shd {} w:fill="{}"/>'.format(nsdecls('w'), fill))
    cell._tc.get_or_add_tcPr().append(shading_elm)

# Populate the header row of the table with DataFrame column names
header_cells = table.rows[0].cells
for i, column_name in enumerate(df.columns):
    header_cells[i].text = column_name
    header_cells[i].paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    run = header_cells[i].paragraphs[0].runs[0]
    run.font.size = Pt(12)  # Set font size for the header
    set_cell_background(header_cells[i], "DDDDDD")  # Set a light gray background for the header

# Populate the table with data from the DataFrame
for i, row in enumerate(df.itertuples(), start=1):
    for j, value in enumerate(row[1:]):  # Skip the index in row
        cell = table.rows[i].cells[j]
        cell.text = str(value)
        cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        for paragraph in cell.paragraphs:
            for run in paragraph.runs:
                run.font.size = Pt(10)  # Set font size for table content

# Define the path for saving the document
doc_path = '/content/Classes_Document.docx'
# Save the document
doc.save(doc_path)
